<a href="https://colab.research.google.com/github/brianp0513/NLP-project/blob/main/LSTM_NLP_Project_version3_with_K_fold_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
import time
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
import os
import tensorflow as tf
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
np.random.seed(1234)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df

In [3]:
def dataPreprocessingForX(df, columnName1):
    df[columnName1] = df[columnName1].map(lambda text: text.lower())
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    stop_words = set(nltk.corpus.stopwords.words('english'))
    df[columnName1] = df[columnName1].map(lambda tokens: [w for w in tokens if not w in stop_words])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
    df[columnName1] = df[columnName1].map(lambda text: re.sub('[^A-Za-z]+', ' ', text))
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    lemmatizer = WordNetLemmatizer()
    df[columnName1] = df[columnName1].map(lambda text: [lemmatizer.lemmatize(i) for i in text])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))

def dataPreprocessingForY(df, columnName2):
    df[columnName2] = df[columnName2].astype('int32')

In [6]:
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 1000)
df = getTextFromFiles(df, data_path_nd, 0, 500)
dataPreprocessingForX(df, 'text')
dataPreprocessingForY(df, 'depression')

In [10]:

def RNN(model):

  model.add(tf.keras.layers.Embedding(input_dim = max_words, output_dim = 8, input_length= max_len))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64 , return_sequences=True)))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(80))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))
  model.add(tf.keras.layers.Activation('sigmoid'))
  # model.build(input_shape=sequences_matrix.shape)

  return model


In [11]:
def RNN2(model):

  model.add(tf.keras.layers.Embedding(input_dim = max_words, output_dim = 10, input_length= max_len))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)))
  model.add(tf.keras.layers.Dense(80))
  model.add(tf.keras.layers.Activation('tanh'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))
  model.add(tf.keras.layers.Activation('sigmoid'))
  # model.build(input_shape=sequences_matrix.shape)
  # model.build(input_shape=sequences_matrix.shape)

  return model


In [13]:
 def CNN_LSTM(model): 
  model.add(tf.keras.layers.Embedding(max_words,50,input_length=max_len))
  model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.LSTM(64))
  # model.add(tf.keras.layers.Dense(256))
  # model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))
  model.add(tf.keras.layers.Activation('sigmoid'))

  return model

# 2 BiLSTM

5-Fold Cross Validation

In [15]:
from sklearn.model_selection import KFold
import numpy as np

# Input Data


X = df['text'].to_numpy()
Y = df['depression'].to_numpy().astype(float)

max_words = 1000
max_len = 150

batch_size = 4
no_epochs = 10
verbosity = 0

num_folds = 5



# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
# inputs = np.concatenate((X_train, X_test), axis=0)
# targets = np.concatenate((Y_train, Y_test), axis=0)

inputs = X
targets = Y

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Data format matching
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(inputs[train])
  sequences = tok.texts_to_sequences(inputs[train])
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

  # Define the model architecture
  model = tf.keras.models.Sequential()
  model = RNN(model)
  # model.summary()
  model.build(input_shape=sequences_matrix.shape)
  # Compile the model
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model.fit(sequences_matrix, targets[train], batch_size=batch_size, epochs=no_epochs, verbose=verbosity)
  
  # Data format matching
  test_sequences = tok.texts_to_sequences(inputs[test])
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  

  # Generate generalization metrics
  # accr = model.evaluate(test_sequences_matrix,Y_test)
  # print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

  scores = model.evaluate(test_sequences_matrix, targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.39350879192352295; accuracy of 87.33333349227905%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.2682547867298126; accuracy of 89.66666460037231%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.2735030949115753; accuracy of 89.33333158493042%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 0.44815099239349365; accuracy of 89.66666460037231%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.29126405715942383; accuracy of 91.00000262260437%
------------------------------------------------------------------------
Score per fold
---------------------------------------

# BiLSTM

In [16]:
from sklearn.model_selection import KFold
import numpy as np

# Input Data


X = df['text'].to_numpy()
Y = df['depression'].to_numpy().astype(float)

max_words = 1000
max_len = 150

batch_size = 4
no_epochs = 10
verbosity = 0

num_folds = 5



# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
# inputs = np.concatenate((X_train, X_test), axis=0)
# targets = np.concatenate((Y_train, Y_test), axis=0)

inputs = X
targets = Y

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Data format matching
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(inputs[train])
  sequences = tok.texts_to_sequences(inputs[train])
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

  # Define the model architecture
  model = tf.keras.models.Sequential()
  model = RNN2(model)
  # model.summary()
  model.build(input_shape=sequences_matrix.shape)
  # Compile the model
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model.fit(sequences_matrix, targets[train], batch_size=batch_size, epochs=no_epochs, verbose=verbosity)
  
  # Data format matching
  test_sequences = tok.texts_to_sequences(inputs[test])
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  

  # Generate generalization metrics
  # accr = model.evaluate(test_sequences_matrix,Y_test)
  # print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

  scores = model.evaluate(test_sequences_matrix, targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.2703160047531128; accuracy of 89.87554907798767%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.34834665060043335; accuracy of 87.79111504554749%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.3335161805152893; accuracy of 89.73778486251831%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 0.33494457602500916; accuracy of 87.5889003276825%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.2337017059326172; accuracy of 90.7977819442749%
------------------------------------------------------------------------
Score per fold
------------------------------------------

# CNN+LSTM

In [17]:
from sklearn.model_selection import KFold
import numpy as np

# Input Data


X = df['text'].to_numpy()
Y = df['depression'].to_numpy().astype(float)

max_words = 1000
max_len = 150

batch_size = 4
no_epochs = 10
verbosity = 0

num_folds = 5



# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
# inputs = np.concatenate((X_train, X_test), axis=0)
# targets = np.concatenate((Y_train, Y_test), axis=0)

inputs = X
targets = Y

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Data format matching
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(inputs[train])
  sequences = tok.texts_to_sequences(inputs[train])
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

  # Define the model architecture
  model = tf.keras.models.Sequential()
  model = CNN_LSTM(model)
  # model.summary()
  model.build(input_shape=sequences_matrix.shape)
  # Compile the model
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model.fit(sequences_matrix, targets[train], batch_size=batch_size, epochs=no_epochs, verbose=verbosity)
  
  # Data format matching
  test_sequences = tok.texts_to_sequences(inputs[test])
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  

  # Generate generalization metrics
  # accr = model.evaluate(test_sequences_matrix,Y_test)
  # print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

  scores = model.evaluate(test_sequences_matrix, targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.31100690364837646; accuracy of 88.99999856948853%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.227126345038414; accuracy of 93.33333373069763%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.495379239320755; accuracy of 86.66666746139526%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 0.38000231981277466; accuracy of 86.33333444595337%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.2107895463705063; accuracy of 92.33333468437195%
------------------------------------------------------------------------
Score per fold
------------------------------------------